## Load Data

In [1]:
# Use all processor cores
from sklearnex import patch_sklearn
patch_sklearn()

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# Load libraries
import jsonlines
import pandas as pd

In [3]:
%%time
# Load JSON data into a list of dictionaries
data = []
with jsonlines.open('categorized-comments.jsonl') as reader:
    for obj in reader.iter(type=dict, skip_invalid=True):
        data.append(obj)

Wall time: 3.47 s


In [4]:
# Convert data to DataFrame
cat_comments_df = pd.DataFrame(data)
cat_comments_df.head()

,cat,txt
0,sports,Barely better than Gabbert? He was significant...
1,sports,Fuck the ducks and the Angels! But welcome to ...
2,sports,Should have drafted more WRs.\n\n- Matt Millen...
3,sports,[Done](https://i.imgur.com/2YZ90pm.jpg)
4,sports,No!! NOO!!!!!


## Preprocess Text

In [5]:
# Load libraries
import re
import sys
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import PorterStemmer

# from sklearn.feature_extraction.text import CountVectorizer

In [6]:
df = cat_comments_df.copy()

In [7]:
# df.head()

In [8]:
punctuation_dict = dict.fromkeys(i for i in range(sys.maxunicode) 
                            if unicodedata.category(chr(i)).startswith('P'))

stop_words = stopwords.words('english')
stopwords_dict = Counter(stop_words)

def cleanText(string):
    '''Processes string and returns cleaned up list of words'''
    
    # Convert to lowercase
    string = string.lower()
    
    # Remove URLs
    string = re.sub(r'http\S+', '', string)
    
    # Remove punctuation
    string = string.translate(punctuation_dict)
    
    # Remove newlines
    string = string.replace("\n", " ")
    
    # Remove stopwords
    string = [word for word in string.split() if word not in stopwords_dict]
    
    return string

In [9]:
%%time
# Clean up the text in the 'txt' column
df.txt = df.txt.apply(lambda string: cleanText(string))

Wall time: 9.22 s


In [10]:
# df.head()

In [11]:
%%time
# Apply PorterStemmer
porter = PorterStemmer()
df['txt_stems'] = df.txt.apply(lambda words: [porter.stem(word) for word in words])

Wall time: 3min 36s


In [12]:
%%time
df['txt_str'] = df.txt_stems.apply(lambda s: ' '.join(map(str, s)))

Wall time: 2.01 s


In [13]:
df.head()

,cat,txt,txt_stems,txt_str
0,sports,"[barely, better, gabbert, significantly, bette...","[bare, better, gabbert, significantli, better,...",bare better gabbert significantli better year ...
1,sports,"[fuck, ducks, angels, welcome, new, niners, fans]","[fuck, duck, angel, welcom, new, niner, fan]",fuck duck angel welcom new niner fan
2,sports,"[drafted, wrs, matt, millen, probably]","[draft, wr, matt, millen, probabl]",draft wr matt millen probabl
3,sports,[done],[done],done
4,sports,[noo],[noo],noo


## Prepare Text for Model-Building

In [14]:
# Load libraries
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import pos_tag
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Back up, let's sample to equal sized groups:
https://stackoverflow.com/questions/41345289/getting-a-random-sample-in-python-dataframe-by-category

In [15]:
cat_group = df.groupby('cat', as_index=False, group_keys=False)

In [16]:
balancedDF = cat_group.apply(lambda s: s.sample(25000, replace=False))

In [17]:
balancedDF.cat.value_counts()

sports                    25000
science_and_technology    25000
video_games               25000
Name: cat, dtype: int64

In [18]:
# text_data, string = [], " "

# for text in balancedDF.txt_stems:
#     text_data.append(string.join(text))

In [19]:
# # Create the bag of words feature matrix
# count = CountVectorizer()
# bag_of_words = count.fit_transform(text_data)

## 20.17 Classifying Text

In [20]:
# Load libraries
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import models
from keras import layers

In [21]:
# Set the number of features we want
number_of_features = 5000

# Set up data and labels
# X = bag_of_words
X = balancedDF.txt_str
y = balancedDF.cat

# Train Test Split
from sklearn.model_selection import train_test_split
data_train, data_test, target_vector_train, target_vector_test = train_test_split(X, y) # Default is 1/4 --> test

In [22]:
data_train[:5]

5620      exactli market almost exclus outsid us yet os ...
208705                  la liga 20 day ago search bar right
236239                      liter forgot women footbal fifa
380505    problem assum sinc ycg ok nintendo doubt ycg g...
329031    ciss world new forev beforehand come rumour sc...
Name: txt_str, dtype: object

In [23]:
# features_train = data_train
# features_test = data_test

In [24]:
from keras.preprocessing.text import Tokenizer
# Convert movie review data to one-hot encoded feature matrix
tokenizer = Tokenizer(num_words=number_of_features)
# tokenizer = Tokenizer(num_words=number_of_features, split=',', char_level=False)
tokenizer.fit_on_texts(X)
features_train = tokenizer.texts_to_matrix(data_train, mode="binary")
features_test = tokenizer.texts_to_matrix(data_test, mode="binary")

In [25]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit(y)
target_train = lb.transform(target_vector_train)
target_test = lb.transform(target_vector_test)

In [26]:
from keras.models import Sequential
# Start neural network
network = Sequential()

In [27]:
from keras import layers
# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=500,
                         activation="relu",
                         input_shape=(number_of_features,)))

# Add fully connected layer with a ReLU activation function
network.add(layers.Dense(units=150, activation="relu"))

# Add fully connected layer with a softmax activation function
network.add(layers.Dense(units=3, activation="softmax"))

In [28]:
# Compile neural network
network.compile(loss="categorical_crossentropy", # Cross-entropy
                optimizer="rmsprop", # Root Mean Square Propagation
                metrics=["accuracy"]) # Accuracy performance metric

In [29]:
# Train neural network
history = network.fit(features_train, # Features
                      target_train, # Target
                      epochs=5, # Three epochs
                      verbose=1, # Some output
                      batch_size=75, # Number of observations per batch
                      validation_data=(features_test, target_test)) # Test data

Epoch 1/5
750/750 [==============================] - 21s 25ms/step - loss: 0.7002 - accuracy: 0.6743 - val_loss: 0.5716 - val_accuracy: 0.7358
Epoch 2/5
750/750 [==============================] - 19s 25ms/step - loss: 0.5099 - accuracy: 0.7674 - val_loss: 0.5640 - val_accuracy: 0.7437
Epoch 3/5
750/750 [==============================] - 18s 24ms/step - loss: 0.4530 - accuracy: 0.7978 - val_loss: 0.5632 - val_accuracy: 0.7418
Epoch 4/5
750/750 [==============================] - 18s 24ms/step - loss: 0.3936 - accuracy: 0.8267 - val_loss: 0.5984 - val_accuracy: 0.7422
Epoch 5/5
750/750 [==============================] - 19s 25ms/step - loss: 0.3337 - accuracy: 0.8527 - val_loss: 0.6915 - val_accuracy: 0.7389


## 20.7 Visualize Training History

In [30]:
# import matplotlib.pyplot as plt
# # Get training and test loss histories
# training_loss = history.history["loss"]
# test_loss = history.history["val_loss"]

# # Create count of the number of epochs
# epoch_count = range(1, len(training_loss) + 1)

# # Visualize loss history
# plt.plot(epoch_count, training_loss, "r--")
# plt.plot(epoch_count, test_loss, "b-")
# plt.legend(["Training Loss", "Test Loss"])
# plt.xlabel("Epoch")
# plt.ylabel("Loss")
# plt.show();

In [31]:
# # Get training and test accuracy histories
# training_accuracy = history.history["accuracy"]
# test_accuracy = history.history["val_accuracy"]
# plt.plot(epoch_count, training_accuracy, "r--")
# plt.plot(epoch_count, test_accuracy, "b-")

# # Visualize accuracy history
# plt.legend(["Training Accuracy", "Test Accuracy"])
# plt.xlabel("Epoch")
# plt.ylabel("Accuracy Score")
# plt.show();

## 20.13 Tuning Neural Networks

In [32]:
# Load libraries
import numpy as np
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_classification

In [33]:
# Create function returning a compiled network
def create_network(optimizer="rmsprop"):
    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=500,
                             activation="relu",
                             input_shape=(number_of_features,)))

    # Add fully connected layer with a ReLU activation function
    network.add(layers.Dense(units=150, activation="relu"))

    # Add fully connected layer with a softmax activation function
    network.add(layers.Dense(units=3, activation="softmax"))

    # Compile neural network
    network.compile(loss="categorical_crossentropy", # Cross-entropy
                    optimizer=optimizer, # Root Mean Square Propagation
#                     optimizer="rmsprop", # Root Mean Square Propagation
                    metrics=["accuracy"]) # Accuracy performance metric

    # Return compiled network
    return network

In [34]:
%%time
# Set random seed
np.random.seed(42)

# Wrap Keras model so it can be used by scikit-learn
neural_network = KerasClassifier(build_fn=create_network, verbose=0)

# Create hyperparameter space 
## Round 1: 3, 50, 'adam' were best
## Round 2: 5, 75, 'rmsprop' were best
epochs =  [3, 5, 7]      #1[1,  3, 10] #2[ 1,  3,  5]
batches = [50, 75, 100] #1[5, 15, 50] #2[30, 50, 75]
optimizers = ["rmsprop", "adam"]

# Create hyperparameter options
hyperparameters = dict(optimizer=optimizers, epochs=epochs, batch_size=batches)

# Create grid search
grid = GridSearchCV(estimator=neural_network, param_grid=hyperparameters)

# Fit grid search
# grid_result = grid.fit(features, target)
# grid_result = grid.fit(features_train, # Features
#                       target_train, # Target
# #                       epochs=150, # Three epochs
#                       verbose=1, # Some output
# #                       batch_size=100, # Number of observations per batch
#                       validation_data=(features_test, target_test)) # Test data
grid_result = grid.fit(features_train, # Features
                      target_train, # Target
#                       epochs=epochs, # Three epochs
                      verbose=1, # Some output
#                       batch_size=batches, # Number of observations per batch
                      validation_data=(features_test, target_test)) # Test data
# grid_result = grid.fit(features_train, # Features
#                       target_train, # Target
#                       epochs=5, # Three epochs
#                       verbose=1, # Some output
#                       batch_size=75, # Number of observations per batch
#                       validation_data=(features_test, target_test)) # Test data

MemoryError: Unable to allocate 1.68 GiB for an array with shape (45000, 5000) and data type float64

In [35]:
# View hyperparameters of best neural network
grid_result.best_params_

NameError: name 'grid_result' is not defined

In [ ]:
# %%time
# # Get predictions
# predictions = grid_result.predict(features_test)

In [ ]:
# %%time
# # Get predictions
# predictions = np.argmax(grid_result.predict(features_test), axis=-1)

### Predictions / Evaluation

In [ ]:
# %%time
# # Get predictions
# predictions = network.predict(features_test)

In [ ]:
%%time
# Get predictions
predictions = network.predict(features_test)
predictions = np.argmax(predictions, axis=1)
y_test = np.argmax(target_test, axis=1)

In [ ]:
# %%time
# # Get predictions
# predictions = np.argmax(network.predict(features_test), axis=-1)

### Evaluate how well our model performed

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

#### Confusion matrix

In [ ]:
cm = confusion_matrix(y_test, predictions)

In [ ]:
print(cm)

In [ ]:
# print(confusion_matrix(target_test, predictions))

#### Precision / Recall / F1 / Suport (Classification report)

In [ ]:
cr = classification_report(y_test, predictions)

In [ ]:
print(cr)

In [ ]:
from win32com.client import Dispatch
speak = Dispatch("SAPI.SpVoice").Speak

In [ ]:
speak("modeling complete")